This file is used to build all of the json data that gets loaded by `train.ipynb`

In [20]:
# Create palette file from palette.png
from util.convert_image_to_palette import convert 

convert("data/palette.png", "data/palette.json")

In [1]:
# Convert all files in the .vox folder to json files
import os
from util.convert_vox_to_json import convert_vox_to_json

filenames = os.listdir('training/vox')

for filename in filenames:
    new_filename = filename.replace("vox", "json")
    convert_vox_to_json("training/vox/" + filename, "training/json/" + new_filename)

In [2]:
# Convert json files to vox files for viewing
import os
from util.convert_vox_to_json import convert_json_to_vox

filenames = os.listdir('training/json')

for filename in filenames:
    new_filename = filename.replace("json", "vox")
    convert_json_to_vox("training/json/" + filename, "training/view_vox/" + new_filename)

In [ ]:
# Convert json files in examples to vox files for viewing
import os
from util.convert_vox_to_json import convert_json_to_vox

filenames = os.listdir('training/json')

for filename in filenames:
    new_filename = filename.replace("json", "vox")
    convert_json_to_vox("examples/json/" + filename, "examples/view_vox/" + new_filename)